### 1. Gérer les données avec *Dataset* et *Dataloader*


Les classes `Dataset` et `Dataloader` permettent de faciliter la gestion des données sous `PyTorch`. 

La classe `Dataset` est une classe abstraite qui permet de préciser comment un exemple est chargé, pré-traité, transformé ... et donne accès par l'intermédiaire d'un itérateur à chaque exemple d'un jeu de donnnées. 

La classe `Dataloader` encapsule un jeu de données et permet de requêter de diverses manières ce jeu de données: spécifier la taille du mini-batch, si l'ordre doit être aléatoire ou non, de quelle manière sont concaténés les exemples etc... Combines a dataset and a sampler and provides an iterable over the given dataset

Pour implémenter un `Dataset`, il suffit de définit 2 méthodes `__getitem__(self, index)` et `__len(self)`:

`from torch.utils.data import Dataset, DataLoader`

`class MonDataset(Dataset):`

    `def __init__(self, ...):`

        `pass`

    `def __getitem__(self, index):`

    retourne un couple (exemple, label) correspondant à l'index
        
        `pass`
    
    `def __len__(self):`

    renvoie la taille du jeu de données
    
        `pass`
    

Principaux avantages (mis à part le prétraitement possible) est qu'il n'est pas nécessaire de charger tout le jeu de données en mémoire, le chargement se fait à la volée. Par ailleurs, la classe pré-existante `TensorDataset` permet de construire un dataset pour une liste de tenseurs passée en argument (le ième exemple est un nuplet composé de la ième ligne de chaque tenseur)

Une fois `MonDataset` implémenté il suffit de créer un `DataLoader` de la manière suivante :

`data = DataLoader(MonDataset(...), shuffle = True, batch_size = BATCH_SIZE)`

création du dataloader en spécifiant la taille du batch et ordre aléatoire

`for x, y in data:`

    x et y est un batch de taille BATCH_SIZE

#### Question 1

Je veux implémenter un dataset pour le jeu de données MNIST qui renvoie une image sous la forme d'un vecteur normalisé entre 0 et 1 et le label associé (sans utiliser `TensorDataset`)

### 2. GPU

Afin de profiter de la puissance de calcul d'un GPU, il faut obligatoirement spécifier à `PyTorch` de charger les tenseurs sur le GPU ainsi que le module (i.e. les paramètres du module). Il n'est pas possible de faire des opérations lorsqu'une partie des tenseurs est sur GPU et l'autre sur CPU (un message d'erreur s'affiche). L'opérateur `to(device)` des tenseurs et des modules permet de les copier sur le GPU (ou CPU) spécifié. Attention l'opération n'est pas *inplace*, il faut sauver le résultat dans une variable qui elle sera sur le bon device.

*Permet de sélectionner le gpu si disponible*

`device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')`

`model = ...`

`model = model.to(device)` *chargement du module sur device*

`x = x.to(device)` *charge données sur device*

`y = model(x)` *calcul gpu

`y = y.to(device = 'cpu)` *si on veut remettre sur cpu*

### 3. Checkpointing

Il est souvent utile de sauvegarder au fur et à mesure de l'apprentissage le modèle afin par exemple de pouvoir reprendre les calculs en cas d'interruption. `Pytorch` a un mécanisme très pratique pour cela par l'intermédiaire de la fonction `state_dict()` qui permet de renvoyer sous forme de dictionnaire les paramètres importants d'un modèle (les paramètres d'apprentissage). Mais il est souvent nécessaire également de connaître l'était de l'optimiseur utilisé pour reprendre les calculs. Cette même fonction `state_dict()` permet également de suaver les valeurs des paramètres importants pour l'optimiseur utilisé. En pratique, les fonctions haut niveau `torch.save()` et `torch.load()` permettent très facilement de sauvegarder et charger les informations voulues et des info complémentaires: elles vont utiliser le sérialiseur usuel de python `pickle` pour les structures habituelles et les fonction `state_dict()` pour les objets de `Pytorch`

*Exemple : modèle d'entraînement utilisant pytorch pour le deeplearning*

`from pathlib import Path`

`savepath = Path("model.pch")`
*#Creation d'un objet 'Path' pr représenter le chemin du fichier de sauvegarde.*
*##Utilisé pr enregistré le modèle entrainé*

`device = torch.Device('cuda' is torch.cuda.is_available() esle 'cpu)`
*#Vérification de la disponibilité de GPU*

*définit classe 'State' pr stocker l'état du modèle et de l'optimiseur*

`class State`:
        
        def __init__(self, model, optim):
                self.model = model
                self.optim = optim
                self.epoch , self.iteration = 0, 0
        
        if savepath.is_file():
                with savepath.open("rb") as fp
                        state = torch.load(fp) 
                        #on recommence depuis le modele sauvegardé
        else : 
                model = 
                model = model.to(device)
                optim = 
                state = State(model, optim)

        for epoch in range (state.epoch, ITERATIONS):
                for x, y in train_loader:
                        x = x.to(device)
                        xhat = state.model(x)
                        l = loss(xhat, x)
                        l.backward()
                        state.optim.step()
                        state.iteration +=1
                with savepath.open("wb") as fp:
                        state.epoch = epoch + 1
                        torch.save(state, fp)
                                

        

### 4. Implémentation d'un autoencoder

Un autoencoder est un réseau de neurones qui permet de projeter (encoder) un jeu de données dans un espace de très petite dimension (il *compresse* le jeu de données). La dimension de sortie est la même que l'entrée, il est entrainé de manière à ce que la sortie soit la plus proche possible que l'entrée $f(x) ~x$ avec un coût aux moindres carrés par exemple ou en coût de cross entropie. On appelle *décodage* le calcul de la sortie à partir des données projetées.